In [1]:
import sys
sys.path.append("src")
import os

In [2]:
import libfeatureselection
import libpybiofeature
from libmodel import t3mm

2023-07-23 14:35:26.488257: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-23 14:35:26.490702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-23 14:35:26.490710: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-23 14:35:27.030909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-23 14:35:27.030922: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [3]:
import numpy as np
import pandas as pd
import json
from Bio import SeqIO

In [4]:
import utils
work_Dir = utils.workdir.workdir(os.getcwd(), 4)
work_Dir

'/mnt/md0/Users/georgezhao/Source/TxSEml_Addon'

In [5]:
protype, cter_bool, db_size = 6, False, 'small'
Tx_arg = {
    "type": f'T{protype}',
    # 'seq_id': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id.json']),
    'shufflesplit_index_file': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id_shufflesplit.json']),
    'fasta': {
        'cter': cter_bool,
        'sp': {
            'p': "data/T6SE/anti-bacterial-effector_p.fasta",
            'n': "data/T6SE/anti-bacterial-effector_n.fasta"
        },
        'nosp': {
            'p': "data/T6SE/anti-eukaryotic-effector_p.fasta",
            'n': "data/T6SE/anti-eukaryotic-effector_n.fasta"
        },
    },
    "ss": {
        'sp': {
            'p': "out/libfeatureselection/A_feature_research/featuredb/bert/ab_p_ss.pkl",
            'n': "out/libfeatureselection/A_feature_research/featuredb/bert/ab_n_ss.pkl",
        },
        'nosp': {
            'p': "out/libfeatureselection/A_feature_research/featuredb/bert/ae_p_ss.pkl",
            'n': "out/libfeatureselection/A_feature_research/featuredb/bert/ae_n_ss.pkl",
        },
    },
    "sa": {
        'sp': {
            'p': "out/libfeatureselection/A_feature_research/featuredb/bert/ab_p_sa.pkl",
            'n': "out/libfeatureselection/A_feature_research/featuredb/bert/ab_n_sa.pkl",
        },
        'nosp': {
            'p': "out/libfeatureselection/A_feature_research/featuredb/bert/ae_p_sa.pkl",
            'n': "out/libfeatureselection/A_feature_research/featuredb/bert/ae_n_sa.pkl",
        },
    },
    'possum': {
        'index': "out/libfeatureselection/A_feature_research/featuredb/possum/possum_index.json",
        'pssm_fdb_pattern': "out/libfeatureselection/A_feature_research/featuredb/possum/{zipid}_pssm_features.zip",
        'pssm_rdb_pattern': "out/libfeatureselection/A_feature_research/featuredb/possum/{zipid}_pssm_files.zip"
    },
    'model': {
        'size': db_size,
        'cter': cter_bool,
        "path_to_save_dir": f"out/libfeatureselection/T6/model/PyPredT6_AB/model/"
    }
}

In [6]:
seq_id_dict = {
    "sp": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['sp']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['sp']['n'], "fasta") ],
    },
    "nosp": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['nosp']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['nosp']['n'], "fasta") ],
    },
}
with open(f"data/T6SE/seq_id.json", "w+", encoding="UTF-8") as f:
    json.dump(
        obj=seq_id_dict,
        fp=f,
    )

In [7]:
feature_data_set = []

In [8]:
feature_data_set.append({
    "name": "AAC",
    "sp_p": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_AAC: 100%|██████████| 33/33 [00:00<00:00, 73467.11it/s]


In [9]:
feature_data_set.append({
    "name": "DAC",
    "sp_p": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_DAC: 100%|██████████| 33/33 [00:00<00:00, 18069.46it/s]


In [10]:
feature_data_set.append({
    "name": "18-PP",
    "sp_p": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_etpp_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_etpp: 100%|██████████| 33/33 [00:00<00:00, 7157.15it/s]


In [11]:
feature_data_set.append({
    "name": "CJ",
    "sp_p": libpybiofeature.featurebuilder.build_conjoint_td_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_conjoint_td_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_conjoint_td_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_conjoint_td_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_CTriad: 100%|██████████| 33/33 [00:00<00:00, 11602.99it/s]


In [12]:
feature_data_set.append({
    "name": "SS-100-Onehot",
    "sp_p": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['ss']['sp']['p'],
        id_list=seq_id_dict['sp']['p'],
        dim=3
    ),
    "sp_n": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['ss']['sp']['n'],
        id_list=seq_id_dict['sp']['n'],
        dim=3
    ),
    "nosp_p": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['ss']['nosp']['p'],
        id_list=seq_id_dict['nosp']['p'],
        dim=3
    ),
    "nosp_n": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['ss']['nosp']['n'],
        id_list=seq_id_dict['nosp']['n'],
        dim=3
    )
})
feature_data_set.append({
    "name": "SA-100-Onehot",
    "sp_p": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['sa']['sp']['p'],
        id_list=seq_id_dict['sp']['p'],
        dim=2
    ),
    "sp_n": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['sa']['sp']['n'],
        id_list=seq_id_dict['sp']['n'],
        dim=2
    ),
    "nosp_p": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['sa']['nosp']['p'],
        id_list=seq_id_dict['nosp']['p'],
        dim=2
    ),
    "nosp_n": libfeatureselection.feature_loader.load_from_bert_onehot(
        path=Tx_arg['sa']['nosp']['n'],
        id_list=seq_id_dict['nosp']['n'],
        dim=2
    )
})

In [13]:
def merge_pn_dataset(
    p_f: pd.DataFrame,
    p_l: np.ndarray,
    n_f: pd.DataFrame,
    n_l: np.ndarray
):
    t_f = pd.concat([p_f, n_f])
    t_l = np.concatenate([p_l, n_l])

    return t_f, t_l

# 合并feature_selected成数据集
data_set_split = {
    datatype: pd.concat([
        item[datatype] for item in feature_data_set
    ], axis=1)
    for datatype in ["sp_p", "sp_n", "nosp_p", "nosp_n"]
}
label_set_split = {
    datatype: np.ones(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["sp_p", "nosp_p",]
} | {
    datatype: np.zeros(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["sp_n", "nosp_n",]
}

t_f, t_l = merge_pn_dataset(
    p_f=data_set_split["sp_p"],
    p_l=label_set_split["sp_p"],
    n_f=data_set_split["sp_n"],
    n_l=label_set_split["sp_n"],
)
v_f, v_l = merge_pn_dataset(
    p_f=data_set_split["nosp_p"],
    p_l=label_set_split["nosp_p"],
    n_f=data_set_split["nosp_n"],
    n_l=label_set_split["nosp_n"],
)

In [14]:
# 丢入model中进行训练
model_path = f"{Tx_arg['model']['path_to_save_dir']}/model/"
os.makedirs(
    model_path,
    exist_ok=True
)
model_path

'out/libfeatureselection/T6/model/PyPredT6_AB/model//model/'

In [15]:
from libfeatureselection import model, model_space
n_jobs = (
    (os.cpu_count() - 2)
    if "n_jobs" not in os.environ or os.environ['n_jobs'] == "" else
    int(os.environ['n_jobs'])
)
n_jobs

30

In [18]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
import math
import functools
import pickle
import gzip
import itertools

from src.libmodel import common, model_optimite

import matplotlib as mpl

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.use14corefonts'] = False
# mpl.rcParams['pdf.usecorefonts'] = True
mpl.rcParams['pdf.compression'] = 9

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'nature'])


from sklearn.metrics import roc_curve, confusion_matrix, precision_score, accuracy_score, f1_score, matthews_corrcoef, auc

def get_evaluation(label: list, pred: list, pro_cutoff: float = None):
    pred = np.nan_to_num(
        pred, copy=True, nan=0.0
    )
    fpr, tpr, thresholds = roc_curve(label, pred)
    if pro_cutoff is None:
        best_one_optimal_idx = np.argmax(tpr - fpr)
        pro_cutoff = thresholds[best_one_optimal_idx]
    pred_l = [1 if i >= pro_cutoff else 0 for i in pred]
    confusion_matrix_1d = confusion_matrix(label, pred_l).ravel()
    confusion_dict = {N: n for N, n in zip(['tn', 'fp', 'fn', 'tp'], list(
        confusion_matrix_1d * 2 / np.sum(confusion_matrix_1d)))}
    evaluation = {
        "accuracy": accuracy_score(label, pred_l),
        "precision": precision_score(label, pred_l),
        "f1_score": f1_score(label, pred_l),
        "mmc": matthews_corrcoef(label, pred_l),
        "rocAUC": auc(fpr, tpr),
        "specificity": confusion_dict['tn'] / (confusion_dict['tn'] + confusion_dict['fp']),
        "sensitivity": confusion_dict['tp'] / (confusion_dict['tp'] + confusion_dict['fn']),
        # "confusion_matrix": confusion_dict,
        # "_roc_Data": {'fpr': list(fpr), 'tpr': list(tpr)},
        'pro_cutoff': pro_cutoff
    }
    return evaluation


def plot_roc_curve(target, pred, path_to_: str):
    fpr, tpr, thresholds = roc_curve(target, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(19.2 / 4, 10.8 / 4))
    plt.axis('square')
    plt.plot(
        fpr, tpr, color='red', lw=2,
        label='ROC curve (area = %0.2f)' % roc_auc
    )
    plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) curve')
    plt.legend(loc="lower right")

    plt.savefig(f"{path_to_}", transparent=True)
    plt.clf()

submodel_choise = None

svm_param_o = [{
    "kernel": ["linear", ],
    "C": [10**i for i in range(-6, 6 + 1)],
}, {
    "kernel": ["poly", "rbf", "sigmoid"],
    "gamma": [10**i for i in range(-6, 6 + 1)],
    "C": [10**i for i in range(-6, 6 + 1)],
}]

rf_param_o = {
    "n_estimators": [2 ** i for i in range(4, 15)],
}

mlp_param_o = {
    "solver": ['adam', 'sgd', 'lbfgs'],
    "activation": ['relu', 'tanh', 'logistic', 'identity'],
    'hidden_layer_sizes': [(100,), (50,), (25,), (10,), ],
    "learning_rate": ['adaptive', 'invscaling', 'constant']
}

knn_optim_o = {
    'n_neighbors': [i for i in range(1, 6)]
}

rf_optim: model_optimite.rf_optim = functools.partial(
    model_optimite.rf_optim,
    default_param={
        'verbose': 0,
        'criterion': 'gini'
    }
)


class PyPredT6_Model(common.Model_Final):
    def __init__(self, cv, desc):
        super().__init__(cv, desc=desc)
        self.model_svm = None
        self.model_nb = None
        self.model_mlp = None
        self.model_knn = None
        self.model_rf = None
        pass

    def tranmodel(self, f, l):
        super().tranmodel(f, l)
        self.model_svm = model_optimite.svm_optim(
            param_o=svm_param_o,
            cv=self.cv,
        ).best_fit(
            f, l, verbose=-1, n_jobs=n_jobs
        )

        self.model_nb = GaussianNB().fit(
            f, l
        )

        self.model_knn = model_optimite.knn_optim(
            param_o=knn_optim_o,
            cv=self.cv
        ).best_fit(
            f, l, verbose=-1, n_jobs=n_jobs
        )

        self.model_rf = rf_optim(
            param_o=rf_param_o,
            cv=self.cv,
        ).best_fit(
            f, l, verbose=-1, n_jobs=n_jobs
        )

        self.model_mlp = model_optimite.mlp_optim(
            param_o=mlp_param_o,
            cv=self.cv,
        ).best_fit(
            f, l, verbose=-1, n_jobs=n_jobs
        )

        return self

    def predict(self, f):
        super().predict(f)
        result = np.stack([
            self.model_svm.predict_proba(f),
            self.model_nb.predict_proba(f)[:, 1],
            self.model_knn.predict_proba(f),
            self.model_rf.predict_proba(f),
            self.model_mlp.predict_proba(f),
        ]).T
        if submodel_choise is None:
            result = (result >= 0.5)
            return result.sum(axis=1) / 5
        if isinstance(submodel_choise, int) == False or submodel_choise < 0 or submodel_choise >= 5:
            raise ValueError(f"Wrong submodel_choise: {submodel_choise}")
        return result[:, submodel_choise]


class PyPredT6_Trainer:
    def __init__(self, ) -> None:
        self.classifier_name = "PyPredT6"
        self.classifier_class = PyPredT6_Model
        self.classifier_param_dict = {
            "desc": "PyPredT6_SP",
            "cv": 5,
        }

        self.model = None
        self.train_best_predicted_pair = None
        self.train_best_5C_predicted_pair = None
        self.best_predicted_pair = None
        self.best_5C_predicted_pair = None
        self.start_to_train_time = datetime.now()
        self.end_of_train_time = None
        pass

    def find_best(
        self,
        X: np.ndarray,
        y: np.ndarray,
        validation: tuple,
        merge_validate: bool
    ):

        self.model = self.classifier_class(
            **self.classifier_param_dict
        )
        self.model.tranmodel(
            pd.DataFrame(X,),
            y,
        )
        self.best_predicted_pair = [
            np.nan_to_num(self.model.predict(
                pd.DataFrame(validation[0],),
            ), nan=0.0),
            validation[1]
        ]
        self.train_best_predicted_pair = [
            np.nan_to_num(self.model.predict(
                pd.DataFrame(X,),
            ), nan=0.0),
            y
        ]

        # 5倍交叉验证
        # 合并数据
        if merge_validate == True:
            full_X = np.concatenate([
                X, validation[0]
            ])
            full_y = np.concatenate([
                y, validation[1]
            ])
        else:
            full_X = np.concatenate([
                X,
            ])
            full_y = np.concatenate([
                y,
            ])

        # 跑模型
        self.best_5C_predicted_pair = []
        self.train_best_5C_predicted_pair = []
        for Kfold_id, (train_id, test_id) in enumerate(
            StratifiedKFold(
                n_splits=5,
                shuffle=True,
                random_state=42
            ).split(full_X, full_y)
        ):

            # 定义模型并加载参数
            fiveC_model = self.classifier_class(
                **self.classifier_param_dict,
            )

            fiveC_model.tranmodel(
                pd.DataFrame(full_X[train_id],),
                full_y[train_id],
            )

            # 预测并记录
            self.best_5C_predicted_pair.append([
                np.nan_to_num(fiveC_model.predict(
                    pd.DataFrame(full_X[test_id],),
                ), nan=0.0),
                full_y[test_id]
            ])
            self.train_best_5C_predicted_pair.append([
                np.nan_to_num(fiveC_model.predict(
                    pd.DataFrame(full_X[train_id],),
                ), nan=0.0),
                full_y[train_id]
            ])

        return self

    def get_summary(self, path_to_dir: str = None):
        os.makedirs(path_to_dir, exist_ok=True)
        model_path = "-"
        if "SAVE_MODEL" in os.environ and os.environ['SAVE_MODEL'] == "1":

            model_path = f"{path_to_dir}/{self.classifier_name}.pkl"
            if path_to_dir is not None:
                with gzip.open(model_path, "wb") as f:
                    pickle.dump(
                        self.grid_search, f
                    )

        model_score_path = f"{path_to_dir}/{self.classifier_name}_score.pkl"
        if path_to_dir is not None:
            with gzip.open(model_score_path, "wb") as f:
                pickle.dump(
                    {
                        "best_predicted_pair": self.best_predicted_pair,
                        "best_5C_predicted_pair": self.best_5C_predicted_pair,
                    }, f
                )
            with gzip.open(model_score_path + ".train", "wb") as f:
                pickle.dump(
                    {
                        "best_predicted_pair": self.train_best_predicted_pair,
                        "best_5C_predicted_pair": self.train_best_5C_predicted_pair,
                    }, f
                )
        else:
            model_score_path = "-"

        plot_roc_curve(
            target=self.best_predicted_pair[1],
            pred=self.best_predicted_pair[0],
            path_to_=f"{path_to_dir}/{self.classifier_name}.pdf"
        )

        model_information = {
            "Classifier_Name": self.classifier_name,
            "Optimitied_Param": dict(),
            "Score": model_score_path,
            "Model_Path": model_path,
            "TimeToStartFit": self.start_to_train_time.strftime("%Y-%m-%d %H:%M:%S")
        }

        training_testing_performance = get_evaluation(
            label=self.best_predicted_pair[1],
            pred=self.best_predicted_pair[0],
        )

        # 计算5C中的平均表现
        FiveFold_result = {}
        for keys in training_testing_performance.keys():
            value_list = []
            for item in self.best_5C_predicted_pair:

                item_performance = get_evaluation(
                    label=item[1],
                    pred=item[0],
                )
                value_list.append(item_performance[keys])

            if keys == "pro_cutoff":
                FiveFold_result[keys] = value_list
            else:
                FiveFold_result[keys] = sum(value_list) / len(value_list)

        self.end_of_train_time = datetime.now()
        model_information["TimeOfSummary"] = self.end_of_train_time.strftime(
            "%Y-%m-%d %H:%M:%S"
        )
        model_information["TimeSpend"] = str(
            self.end_of_train_time - self.start_to_train_time
        )

        return model_information, training_testing_performance, FiveFold_result

In [19]:
PyPredT6_Trainer().find_best(
    X=t_f.values,
    y=t_l,
    validation=(
        v_f.values,
        v_l,
    ),
    merge_validate=False
).get_summary(
    path_to_dir="out/libfeatureselection/T6/PyPredT6_AB/"
)

({'Classifier_Name': 'PyPredT6',
  'Optimitied_Param': {},
  'Score': 'out/libfeatureselection/T6/PyPredT6_AB//PyPredT6_score.pkl',
  'Model_Path': '-',
  'TimeToStartFit': '2023-07-23 14:37:46',
  'TimeOfSummary': '2023-07-23 14:39:59',
  'TimeSpend': '0:02:12.381083'},
 {'accuracy': 0.7727272727272727,
  'precision': 0.78125,
  'f1_score': 0.7692307692307692,
  'mmc': 0.5457051563317492,
  'rocAUC': 0.8521579430670341,
  'specificity': 0.7878787878787878,
  'sensitivity': 0.7575757575757576,
  'pro_cutoff': 0.8},
 {'accuracy': 0.7744588744588745,
  'precision': 0.8305555555555555,
  'f1_score': 0.7355555555555555,
  'mmc': 0.5588779157661717,
  'rocAUC': 0.7849586776859504,
  'specificity': 0.8690909090909091,
  'sensitivity': 0.6745454545454544,
  'pro_cutoff': [0.6, 0.6, 0.8, 0.8, 0.6]})

<Figure size 480x270 with 0 Axes>

In [20]:
PyPredT6_Trainer().find_best(
    X=v_f.values,
    y=v_l,
    validation=(
        t_f.values,
        t_l,
    ),
    merge_validate=False
).get_summary(
    path_to_dir="out/libfeatureselection/T6/PyPredT6_AE/"
)

({'Classifier_Name': 'PyPredT6',
  'Optimitied_Param': {},
  'Score': 'out/libfeatureselection/T6/PyPredT6_AE//PyPredT6_score.pkl',
  'Model_Path': '-',
  'TimeToStartFit': '2023-07-23 14:39:59',
  'TimeOfSummary': '2023-07-23 14:41:41',
  'TimeSpend': '0:01:42.216194'},
 {'accuracy': 0.7547169811320755,
  'precision': 0.7647058823529411,
  'f1_score': 0.7499999999999999,
  'mmc': 0.5097970656333187,
  'rocAUC': 0.7988608045567818,
  'specificity': 0.7735849056603774,
  'sensitivity': 0.7358490566037735,
  'pro_cutoff': 0.8},
 {'accuracy': 0.7417582417582417,
  'precision': 0.7666666666666667,
  'f1_score': 0.752822966507177,
  'mmc': 0.5155594045395453,
  'rocAUC': 0.708843537414966,
  'specificity': 0.6904761904761905,
  'sensitivity': 0.7857142857142858,
  'pro_cutoff': [0.6, 0.4, 0.6, 1.0, 0.8]})

<Figure size 480x270 with 0 Axes>